## 1. Setup
Install dependencies and clone TensorFlow Object Detection API

In [1]:
%tensorflow_version 2.x
import tensorflow as tf
print(tf.__version__)

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
2.17.1


In [2]:
!pip install ensemble-boxes==1.0.6

In [3]:
!pip install -U tf2onnx==1.8.5

In [4]:
!pip install nnoir-onnx==1.0.13

In [5]:
!wget https://raw.githubusercontent.com/amikelive/coco-labels/master/coco-labels-paper.txt
!wget http://download.tensorflow.org/models/object_detection/ssdlite_mobilenet_v2_coco_2018_05_09.tar.gz -P /content/
!tar xfz /content/ssdlite_mobilenet_v2_coco_2018_05_09.tar.gz

--2024-12-06 09:24:38--  https://raw.githubusercontent.com/amikelive/coco-labels/master/coco-labels-paper.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 702 [text/plain]
Saving to: ‘coco-labels-paper.txt.1’

coco-labels-paper.t 100%[===================>]     702  --.-KB/s    in 0s      

2024-12-06 09:24:38 (33.9 MB/s) - ‘coco-labels-paper.txt.1’ saved [702/702]

--2024-12-06 09:24:38--  http://download.tensorflow.org/models/object_detection/ssdlite_mobilenet_v2_coco_2018_05_09.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 172.217.12.27, 142.250.65.123, 172.217.15.251, ...
Connecting to download.tensorflow.org (download.tensorflow.org)|172.217.12.27|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 51025348

In [6]:
!git clone https://github.com/tensorflow/models.git
%cd models/
!git checkout 0ea84d6df6220552cd5229a0181c9e420f815408

Cloning into 'models'...
remote: Enumerating objects: 98785, done.
remote: Counting objects: 100% (1291/1291), done.
remote: Compressing objects: 100% (672/672), done.
remote: Total 98785 (delta 706), reused 1071 (delta 591), pack-reused 97494 (from 1)
Receiving objects: 100% (98785/98785), 622.31 MiB | 21.18 MiB/s, done.
Resolving deltas: 100% (71690/71690), done.
/content/models
Note: switching to '0ea84d6df6220552cd5229a0181c9e420f815408'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

HEAD is now at 0ea84d6df Changing th

In [ ]:
import os
import math
import subprocess
import onnx
import numpy as np
import tensorflow as tf
import cv2
import nnoir
from collections import namedtuple
from PIL import Image, ImageDraw, ImageFont
from ensemble_boxes import *

In [7]:
%cd /content/models/research

/content/models/research


In [8]:
!apt --quiet update
!apt install --quiet protobuf-compiler

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,192 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,525 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-sec

In [9]:
!protoc object_detection/protos/*.proto --python_out=.
!cp object_detection/packages/tf1/setup.py .
!python3 -m pip install .

Processing /content/models/research
  Preparing metadata (setup.py) ... done
  Created wheel for object_detection: filename=object_detection-0.1-py3-none-any.whl size=1628877 sha256=2856ebec449d37b948f5a424e3c45cf28352383f44c85eeadc15e852005f2b0a
  Stored in directory: /tmp/pip-ephem-wheel-cache-bz5piux_/wheels/53/dd/70/2de274d6c443c69d367bd6a5606f95e5a6df61aacf1435ec0d
Successfully built object_detection


In [10]:
!python3 object_detection/builders/model_builder_tf1_test.py

2024-12-06 09:25:32.402614: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-06 09:25:32.443170: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-06 09:25:32.455068: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-06 09:25:34.105632: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Traceback (most recent call last):
  File "/content/models/research/object_detection/builders/model_builder_tf1_test.py", line 21, in <module>
    from object_detection.builders import model_builder
  File "/usr/local/lib/python3.10/dist-pac

## 2. Create TensorFlow Lite Model
set add_postprocessing_op=False

In [11]:
!python /content/models/research/object_detection/export_tflite_ssd_graph.py \
--pipeline_config_path=/content/ssdlite_mobilenet_v2_coco_2018_05_09/pipeline.config \
--trained_checkpoint_prefix=/content/ssdlite_mobilenet_v2_coco_2018_05_09/model.ckpt \
--output_directory=/content \
--add_postprocessing_op=False

2024-12-06 09:25:45.428996: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-06 09:25:45.462044: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-06 09:25:45.472410: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-06 09:25:48.152316: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Traceback (most recent call last):
  File "/content/models/research/object_detection/export_tflite_ssd_graph.py", line 97, in <module>
    from object_detection import export_tflite_ssd_graph_lib
  File "/usr/local/lib/python3.10/dist-packag

In [12]:
!tflite_convert \
--graph_def_file=/content/tflite_graph.pb \
--output_file=/content/ssdlite_mobilenet_v2.tflite \
--input_format=TENSORFLOW_GRAPHDEF \
--output_format=TFLITE \
--inference_type=FLOAT \
--input_shapes=1,300,300,3 \
--input_arrays=normalized_input_image_tensor \
--output_arrays=raw_outputs/box_encodings,raw_outputs/class_predictions

2024-12-06 09:25:54.665400: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-06 09:25:54.702524: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-06 09:25:54.713347: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-06 09:25:58.580265: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
usage: tflite_convert [-h] --output_file OUTPUT_FILE
                      [--saved_model_dir SAVED_MODEL_DIR | --keras_model_file KERAS_MODEL_FILE]
                      [--saved_model_tag_set SAVED_MODEL_TAG_SET]
                      [--s

## 3. Create anchor
- https://qiita.com/PINTO/items/1312d308b553362a8ebf#%EF%BC%96appendix

In [13]:
from tensorflow.python.platform import gfile
from tensorflow.python.framework import tensor_util

GRAPH_PB_PATH = '/content/tflite_graph.pb'

with tf.Session() as sess:
  with gfile.FastGFile(GRAPH_PB_PATH,'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
    sess.graph.as_default()
    tf.import_graph_def(graph_def, name='')
    graph_nodes=[n for n in graph_def.node]
    wts = [n for n in graph_nodes if n.op=='Const']
    for n in wts:
        if n.name == 'anchors':
            print("Name of the node - %s" % n.name)
            print("Value - ")
            anchors = tensor_util.MakeNdarray(n.attr['value'].tensor)
            print("anchors.shape =", anchors.shape)
            print(anchors)
            np.save('/content/anchors.npy', anchors)
            np.savetxt('/content/anchors.csv', anchors, delimiter=',')
            break

AttributeError: module 'tensorflow' has no attribute 'Session'

https://github.com/ChrystleMyrnaLobo/tflite-object-detection/blob/master/inference.py

## 4. Convert TensorFlow Lite Model to ONNX

In [ ]:
!python -m tf2onnx.convert --tflite /content/ssdlite_mobilenet_v2.tflite \
--output /content/ssdlite_mobilenet_v2.onnx --opset 13

In [ ]:
onnx_model = onnx.load("/content/ssdlite_mobilenet_v2.onnx")
try:
    onnx.checker.check_model(onnx_model)
except onnx.checker.ValidationError as e:
    print('The model is invalid: %s' % e)
else:
    print('The model is valid.')

In [ ]:
onnx_model_output =[node.name for node in onnx_model.graph.output]
print(onnx_model_output)

In [ ]:
input_all = [node.name for node in onnx_model.graph.input]
input_initializer =  [node.name for node in onnx_model.graph.initializer]
net_feed_input = list(set(input_all)  - set(input_initializer))

print(net_feed_input)

## 5. Convert ONNX Model to nnoir

In [ ]:
!onnx2nnoir -o /content/ssdlite_mobilenet_v2.nnoir /content/ssdlite_mobilenet_v2.onnx --graph_name ssdlite_mobilenet_v2

In [ ]:
nnoir_model = nnoir.load('/content/ssdlite_mobilenet_v2.nnoir')

In [ ]:
image = cv2.imread("/content/models/research/object_detection/test_images/image1.jpg")
image = cv2.resize(image, (300, 300))
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image_expanded = np.expand_dims(image_rgb.astype(np.float32), axis=0)
image_normalized =  (np.float32(image_expanded) - 127.5) / 127.5

## 6. nnoir Inference

In [ ]:
box_encodings, class_predictions = nnoir_model.run(image_normalized)

In [ ]:
print(box_encodings)

In [ ]:
print(class_predictions)

In [ ]:
labels = np.array([], dtype=int)
result_labels = np.append(labels, np.argmax(class_predictions[0], axis=1))

In [ ]:
# Get Indexes that predict something other than background.
indexes_not_background = np.where(result_labels!=0)

In [ ]:
labels_not_background = np.take(result_labels, indexes_not_background)

In [ ]:
scores = []
scores.append(list(map(max, class_predictions[0])))
scores_not_background = np.take(scores[0], indexes_not_background)

In [ ]:
# from pipeline.config
y_scale = 10.0
x_scale = 10.0
h_scale = 5.0
w_scale = 5.0

def decode_box_encodings(box_encoding, anchors, num_boxes):
    decoded_boxes = np.zeros((num_boxes, 4), dtype=np.float32)
    for i in range(num_boxes):
        ycenter = box_encoding[i][0] / y_scale * anchors[i][2] + anchors[i][0]
        xcenter = box_encoding[i][1] / x_scale * anchors[i][3] + anchors[i][1]
        half_h = 0.5 * math.exp((box_encoding[i][2] / h_scale)) * anchors[i][2]
        half_w = 0.5 * math.exp((box_encoding[i][3] / w_scale)) * anchors[i][3]
        decoded_boxes[i][0] = (xcenter - half_w) # xmin
        decoded_boxes[i][1] = (ycenter - half_h) # ymin
        decoded_boxes[i][2] = (xcenter + half_w) # xmax
        decoded_boxes[i][3] = (ycenter + half_h) # ymax
    return decoded_boxes

In [ ]:
anchors = np.load('/content/anchors.npy')
box_decoded = decode_box_encodings(box_encodings[0], anchors, 1917)
box_not_background = np.take(box_decoded, indexes_not_background, axis=0)
box_clipped = box_not_background.clip(0, 1)

In [ ]:
nms_boxes, nms_scores, nms_labels = nms(box_clipped.tolist(), scores_not_background.tolist(), labels_not_background.tolist(), iou_thr=0.5)

In [ ]:
image_pil = Image.fromarray(image_rgb)
image_pil

In [ ]:
coco_labels = ['background']
with open("/content/coco-labels-paper.txt",'r') as f:
  for line in f:
    coco_labels.append(line.rstrip())

In [ ]:
coco_labels_length = len(coco_labels)
draw = ImageDraw.Draw(image_pil)
for box, score, label in zip(nms_boxes, nms_scores, nms_labels):
  if score> 0.5 and label > 0 and label < coco_labels_length:
    xmin = box[0]*300
    ymin = box[1]*300
    xmax = box[2]*300
    ymax = box[3]*300
    draw.rectangle((xmin, ymin, xmax, ymax), outline=(0, 0, 255), width=2)
    draw.text([xmin+2, ymin+2], text=coco_labels[label])

In [ ]:
image_pil